# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [107]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import os
from api_keys import gkey
import requests
import json



In [108]:
# Import gkey key
gmaps.configure(gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [109]:
city_list_file = "cityDataFile.csv"
city_list_file_df = pd.read_csv(city_list_file)
city_list_file_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bocaiuva,-17.11,-43.81,82.40,24,0,12.75,BR,1599946895
1,1,penzance,50.12,-5.54,59.00,100,86,6.93,GB,1599946593
2,2,manyana,-23.40,21.72,59.90,31,0,8.37,BW,1599946895
3,3,torbay,47.67,-52.73,57.00,54,20,16.11,CA,1599946524
4,4,hithadhoo,-0.60,73.08,81.68,73,25,7.29,MV,1599946519
...,...,...,...,...,...,...,...,...,...,...
561,561,nanortalik,60.14,-45.24,38.82,73,2,3.18,GL,1599946582
562,562,bonthe,7.53,-12.51,76.24,92,95,5.55,SL,1599946518
563,563,jamestown,42.10,-79.24,75.00,73,90,9.17,US,1599946460
564,564,luena,-11.78,19.92,63.46,30,29,5.21,AO,1599947007


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [110]:
locations = city_list_file_df[["Lat", "Lng"]]
humidity = city_list_file_df["Humidity"].astype(float)

In [111]:
fig = gmaps.figure()

# Create heat layer
humidity_heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(humidity_heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [112]:
maxlow_locations_df = city_list_file_df[city_list_file_df['Max Temp'] > 70] 
maxhigh_locations_df = maxlow_locations_df[maxlow_locations_df['Max Temp'] <85]
desired_wind_df = maxhigh_locations_df[maxhigh_locations_df['Wind Speed'] <10]
vacation_temps_df = desired_wind_df[desired_wind_df['Cloudiness'] <10]
vacation_temps_df.dropna(how="any")
vacation_temps_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,vera cruz,-12.63,-41.03,72.41,50,5,9.91,BR,1599946870
38,38,mizdah,31.45,12.98,75.02,69,0,8.41,LY,1599946578
56,56,posse,-14.09,-46.37,76.33,28,0,8.25,BR,1599946861
66,66,cayenne,4.93,-52.33,84.20,70,2,5.82,GF,1599946705
93,93,silistea,44.38,25.35,72.00,60,0,2.24,RO,1599946914
135,135,cayenne,4.93,-52.33,84.20,70,2,5.82,GF,1599946705
137,137,filiatra,37.15,21.58,77.00,78,0,3.69,GR,1599946925
154,154,tabas,33.60,56.92,77.00,16,0,4.70,IR,1599946928
163,163,cayenne,4.93,-52.33,84.20,70,2,5.82,GF,1599946705
169,169,lagoa,39.05,-27.98,71.02,71,1,3.83,PT,1599946565


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [113]:
vacation_temps_df["Hotel Name"]= ""
vacation_temps_df
vacation_temps_nodupes_df= vacation_temps_df.drop_duplicates(subset=['City'])
vacation_temps_nodupes_df

C:\Users\Maher\anaconda3\envs\bcs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,25,vera cruz,-12.63,-41.03,72.41,50,5,9.91,BR,1599946870,
38,38,mizdah,31.45,12.98,75.02,69,0,8.41,LY,1599946578,
56,56,posse,-14.09,-46.37,76.33,28,0,8.25,BR,1599946861,
66,66,cayenne,4.93,-52.33,84.20,70,2,5.82,GF,1599946705,
93,93,silistea,44.38,25.35,72.00,60,0,2.24,RO,1599946914,
137,137,filiatra,37.15,21.58,77.00,78,0,3.69,GR,1599946925,
154,154,tabas,33.60,56.92,77.00,16,0,4.70,IR,1599946928,
169,169,lagoa,39.05,-27.98,71.02,71,1,3.83,PT,1599946565,
183,183,tukrah,32.53,20.58,74.53,71,0,9.69,LY,1599946935,
186,186,axim,4.87,-2.24,75.11,90,0,8.30,GH,1599946935,


In [116]:
target_search = "Hotel"
target_radius = 10000
target_type = "lodging"

# set up a parameters dictionary


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in vacation_temps_nodupes_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    target_coordinates =', '.join(map(str, (lat, lon)))   
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    status = hotels_data["status"]
    if status == "OK":
        hotel = hotels_data["results"][0]["name"]
        vacation_temps_nodupes_df.loc[index, 'Hotel Name'] = hotel        
        
vacation_temps_nodupes_df

# run a request using our params dictionary

#print(json.dumps(hotels_data, indent=4, sort_keys=True))
#hotel = hotels_data["results"][0]["name"]
#print(hotel)

    

C:\Users\Maher\anaconda3\envs\bcs\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,25,vera cruz,-12.63,-41.03,72.41,50,5,9.91,BR,1599946870,
38,38,mizdah,31.45,12.98,75.02,69,0,8.41,LY,1599946578,فندق مزدة الكبير
56,56,posse,-14.09,-46.37,76.33,28,0,8.25,BR,1599946861,Pousada das Palmeiras
66,66,cayenne,4.93,-52.33,84.20,70,2,5.82,GF,1599946705,Royal Amazonia
93,93,silistea,44.38,25.35,72.00,60,0,2.24,RO,1599946914,
137,137,filiatra,37.15,21.58,77.00,78,0,3.69,GR,1599946925,Edem Suites
154,154,tabas,33.60,56.92,77.00,16,0,4.70,IR,1599946928,Amir hotel
169,169,lagoa,39.05,-27.98,71.02,71,1,3.83,PT,1599946565,Graciosa Resort - Biosphere Island Hotel
183,183,tukrah,32.53,20.58,74.53,71,0,9.69,LY,1599946935,النوران برسس
186,186,axim,4.87,-2.24,75.11,90,0,8.30,GH,1599946935,Ankobra Beach


In [123]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_temps_nodupes_df.iterrows()]
locations = vacation_temps_nodupes_df[["Lat", "Lng"]]


In [125]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))